# DS Market store -- Feature Engineering

In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import multiprocessing as mp
import gc
import datetime
from sklearn.preprocessing import LabelEncoder
import calendar
from scipy.sparse import csr_matrix,hstack
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from tqdm import tqdm
import pickle

# Montando Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Reading up the dataframes
df_final=pd.read_csv('/content/drive/MyDrive/ARQUIVOS TFM/NOVOS DF/df_final.csv')
test=pd.read_csv('/content/drive/MyDrive/ARQUIVOS TFM/NOVOS DF/df_final_test.csv')
final_test=pd.read_csv('/content/drive/MyDrive/ARQUIVOS TFM/NOVOS DF/df_final_futuro.csv')

In [4]:
# Usar expressão regular para extrair números da string
df_final['d'] = df_final['d'].str.extract('(\d+)').astype(int)

In [5]:
# Usar expressão regular para extrair números da string
test['d'] = test['d'].str.extract('(\d+)').astype(int)

In [6]:
# Usar expressão regular para extrair números da string
final_test['d'] = final_test['d'].str.extract('(\d+)').astype(int)

# Combinando 36 ultimos dias do df_final com o test para poder adicionar os atrasos de ate 36 dias.

In [7]:
window = 36  # Define a janela de dias que você quer considerar

# Definindo os limites para pegar os últimos 36 dias do df_final
start_d = 1886 - window  # 1886 - 36 = 1850
end_d = 1885  # O último valor de 'd' no df_final

# Pegando os registros do df_final que estão dentro desta janela
last_36_days = df_final[df_final['d'].between(start_d, end_d)]

# Concatenando os últimos 36 dias do df_final com test
combined_data = pd.concat([last_36_days, test]).reset_index(drop=True)





In [8]:
test = combined_data

## Label Encoding df_final

In [9]:
# Converter as colunas relevantes para categóricas
df_final['id'] = df_final['id'].astype('category')
df_final['item'] = df_final['item'].astype('category')
df_final['department'] = df_final['department'].astype('category')
df_final['category'] = df_final['category'].astype('category')
df_final['store'] = df_final['store'].astype('category')
df_final['store_code'] = df_final['store_code'].astype('category')
df_final['event'] = df_final['event'].astype('category')
df_final['region'] = df_final['region'].astype('category')
df_final['weekday'] = df_final['weekday'].astype('category')

# Armazene as categorias juntamente com seus códigos
d_id = dict(zip(df_final['id'].cat.codes, df_final['id']))
d_item_id = dict(zip(df_final['item'].cat.codes, df_final['item']))
d_dept_id = dict(zip(df_final['department'].cat.codes, df_final['department']))
d_cat_id = dict(zip(df_final['category'].cat.codes, df_final['category']))
d_store_id = dict(zip(df_final['store'].cat.codes, df_final['store']))
d_state_id = dict(zip(df_final['region'].cat.codes, df_final['region']))
d_store_code_id = dict(zip(df_final['store_code'].cat.codes, df_final['store_code']))
d_event_id = dict(zip(df_final['event'].cat.codes, df_final['event']))
d_weekday_id = dict(zip(df_final['weekday'].cat.codes, df_final['weekday']))

In [10]:
directory_path = "/content/drive/MyDrive/ARQUIVOS TFM/mapeamento_lojas"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [11]:
import pickle

def save_mapping(mapping, filename):
    with open(os.path.join(directory_path, filename), "wb") as f:
        pickle.dump(mapping, f)

save_mapping(d_id, "d_id.pkl")
save_mapping(d_item_id, "d_item_id.pkl")
save_mapping(d_dept_id, "d_dept_id.pkl")
save_mapping(d_cat_id, "d_cat_id.pkl")
save_mapping(d_store_id, "d_store_id.pkl")
save_mapping(d_state_id, "d_state_id.pkl")
save_mapping(d_store_code_id, "d_store_code_id.pkl")
save_mapping(d_event_id, "d_event_id.pkl")
save_mapping(d_weekday_id, "d_weekday_id.pkl")


## Convertendo dados categóricos para códigos

In [12]:
# Crie listas com nomes de colunas e tipos de dados
cols = df_final.dtypes.index.tolist()
types = df_final.dtypes.values.tolist()

# Itere pelas colunas e, se for do tipo 'category', converta em códigos
for i, tipo in enumerate(types):
    if tipo.name == 'category':
        df_final[cols[i]] = df_final[cols[i]].cat.codes

O método `cat.codes()` retorna uma série com os códigos das categorias da coluna categórica. Os códigos são números inteiros que representam a ordem das categorias.

O `cat.codes()` é útil para codificar dados categóricos para uso em modelos de machine learning. Isso ocorre porque os modelos de machine learning geralmente funcionam melhor com dados numéricos.

Principais pontos

* O `cat.codes()` retorna uma série com os códigos das categorias.
* Os códigos são números inteiros que representam a ordem das categorias.
* A ordem das categorias não é significativa para o `cat.codes()`.
* O `cat.codes()` é útil para codificar dados categóricos para uso em modelos de machine learning.

## Introduzir atrasos

Os recursos de atraso são a maneira clássica pela qual os problemas de previsão de séries temporais são transformados em problemas de aprendizagem supervisionada.

In [13]:
# Introduzir atrasos
lags = [1, 2, 3, 6, 12, 24, 36]
for lag in lags:
    df_final[f'sold_lag_{lag}'] = df_final.groupby(['id', 'item', 'department', 'category', 'store', 'store_code', 'region'], as_index=False)['sold_units'].shift(lag).astype(np.float16)


In [9]:
df_final.head()

,id,item,category,department,store,store_code,region,d,sold_units,date,...,year,month,day,sold_lag_1,sold_lag_2,sold_lag_3,sold_lag_6,sold_lag_12,sold_lag_24,sold_lag_36
0,3,0,0,0,2,3,1,d_1,0,2011-01-29,...,2011,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,1,0,0,2,3,1,d_1,0,2011-01-29,...,2011,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23,2,0,0,2,3,1,d_1,0,2011-01-29,...,2011,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33,3,0,0,2,3,1,d_1,0,2011-01-29,...,2011,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43,4,0,0,2,3,1,d_1,0,2011-01-29,...,2011,1,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id='F3'><h2>Codificação de Médias</h2></a>


Na perspectiva matemática, a codificação de médias reflete a probabilidade da variável-alvo condicional a cada valor da característica. De uma maneira peculiar, essa técnica integra a variável-alvo em sua representação codificada. As codificações de médias foram derivadas com base em um conjunto de características lógicas que consideramos cuidadosamente, segue:
- item
- estado
- loja
- categoria
- departamento
- categoria e departamento
- loja e item
- categoria e item
- departamento e item
- estado e loja
- estado, loja e categoria
- loja, categoria e departamento





In [14]:
# Média de vendas por item
df_final['item_sold_avg'] = df_final.groupby('item')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por estado
df_final['state_sold_avg'] = df_final.groupby('region')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por loja
df_final['store_sold_avg'] = df_final.groupby('store_code')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por categoria
df_final['cat_sold_avg'] = df_final.groupby('category')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por departamento
df_final['dept_sold_avg'] = df_final.groupby('department')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por categoria e departamento
df_final['cat_dept_sold_avg'] = df_final.groupby(['category', 'department'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por loja e item
df_final['store_item_sold_avg'] = df_final.groupby(['store_code', 'item'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por categoria e item
df_final['cat_item_sold_avg'] = df_final.groupby(['category', 'item'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por departamento e item
df_final['dept_item_sold_avg'] = df_final.groupby(['department', 'item'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por estado e loja
df_final['state_store_sold_avg'] = df_final.groupby(['region', 'store_code'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por estado, loja e categoria
df_final['state_store_cat_sold_avg'] = df_final.groupby(['region', 'store_code', 'category'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por loja, categoria e departamento
df_final['store_cat_dept_sold_avg'] = df_final.groupby(['store_code', 'category', 'department'])['sold_units'].transform('mean').astype(np.float16)


## Cálculo das Médias Móveis de Unidades Vendidas para Diferentes Intervalos de Tempo

In [15]:
import numpy as np

window_sizes = [7, 14, 21, 28]

for window_size in window_sizes:
    rolling_col_name = f'rolling_sold_mean_{window_size}'
    df_final[rolling_col_name] = df_final.groupby(['id', 'item', 'category', 'department', 'store', 'store_code', 'region'])['sold_units'].transform(lambda x: x.rolling(window=window_size).mean()).astype(np.float16)


In [18]:
df_final.head()

,id,item,category,department,store,store_code,region,d,sold_units,date,...,state_store_sold_avg,state_store_cat_sold_avg,store_cat_dept_sold_avg,rolling_sold_mean_7,rolling_sold_mean_14,rolling_sold_mean_21,rolling_sold_mean_28,expanding_sold_units_mean,expanding_revenue_mean,selling_trend
0,3,0,0,0,2,3,1,d_1,0,2011-01-29,...,1.317383,0.809082,1.030273,NaN,NaN,NaN,NaN,NaN,NaN,-0.303955
1,13,1,0,0,2,3,1,d_1,0,2011-01-29,...,1.317383,0.809082,1.030273,NaN,NaN,NaN,NaN,NaN,NaN,-0.260498
2,23,2,0,0,2,3,1,d_1,0,2011-01-29,...,1.317383,0.809082,1.030273,NaN,NaN,NaN,NaN,NaN,NaN,-0.144287
3,33,3,0,0,2,3,1,d_1,0,2011-01-29,...,1.317383,0.809082,1.030273,NaN,NaN,NaN,NaN,NaN,NaN,-1.716797
4,43,4,0,0,2,3,1,d_1,0,2011-01-29,...,1.317383,0.809082,1.030273,NaN,NaN,NaN,NaN,NaN,NaN,-0.960938


## Previsão de vendas usando médias móveis

In [16]:
# Definindo as colunas para calcular as médias móveis exponenciais acumuladas
columns_to_calculate = ['sold_units', 'revenue']

# Calculando as médias móveis exponenciais acumuladas
for column in columns_to_calculate:
    df_final[f'expanding_{column}_mean'] = df_final.groupby(['id', 'item', 'department', 'category', 'store', 'store_code', 'region'])[column].transform(lambda x: x.expanding(2).mean()).astype(np.float16)

## Tendência


Vou criar uma característica chamada "tendência de vendas", que terá um valor positivo se as vendas diárias de itens forem maiores do que a média de todo o período (d_1 - d_1913), caso contrário, será negativo. É possível adicionar mais características de tendência, mas vou adicionar apenas essa para mantê-la simples.

Em outras palavras, a "tendência de vendas" mostrará se as vendas diárias estão acima ou abaixo da média global do período total. Se estiverem acima, terá um valor positivo; se estiverem abaixo, terá um valor negativo. Isso pode ajudar a identificar se as vendas estão tendendo a ser melhores ou piores do que a média histórica.

In [17]:
# Calcula a média diária de vendas para cada combinação de id, item_id, dept_id, cat_id, store_id, state_id e d
df_final['daily_avg_sold'] = df_final.groupby(['id', 'item', 'department', 'category', 'store', 'store_code', 'region', 'd'])['sold_units'].transform('mean').astype(np.float16)

# Calcula a média de vendas geral para cada combinação de id, item_id, dept_id, cat_id, store_id e state_id
df_final['avg_sold'] = df_final.groupby(['id', 'item', 'department', 'category', 'store', 'store_code', 'region'])['sold_units'].transform('mean').astype(np.float16)

# Calcula a tendência de vendas subtraindo a média diária de vendas da média de vendas geral
df_final['selling_trend'] = (df_final['daily_avg_sold'] - df_final['avg_sold']).astype(np.float16)

# Remove as colunas daily_avg_sold e avg_sold
df_final.drop(['daily_avg_sold', 'avg_sold'], axis=1, inplace=True)


# Salvando novo df_final encoded

Agora, como todos os novos recursos foram criados, vamos salvar os dados para que possam ser treinados separadamente. Além disso, os atrasos introduzem muitos valores nulos, então removerei os dados dos primeiros 36 dias, pois introduzi atrasos de até 36 dias.

In [14]:
#verificando valores em falta
soma_missings_por_coluna = df_final.isnull().sum()
soma_missings_por_coluna

id                                 0
item                               0
category                           0
department                         0
store                              0
store_code                         0
region                             0
d                                  0
sold_units                         0
date                               0
weekday                            0
weekday_int                        0
event                              0
price                              0
revenue                            0
year                               0
month                              0
day                                0
sold_lag_1                     30490
sold_lag_2                     60980
sold_lag_3                     91470
sold_lag_6                    182940
sold_lag_12                   365880
sold_lag_24                   731760
sold_lag_36                  1097640
item_sold_avg                      0
state_sold_avg                     0
s

In [19]:
# Now, filter the DataFrame
df_final = df_final[df_final['d']>=37]


In [16]:
#verificando valores em falta
soma_missings_por_coluna = df_final.isnull().sum()
soma_missings_por_coluna

id                           0
item                         0
category                     0
department                   0
store                        0
store_code                   0
region                       0
d                            0
sold_units                   0
date                         0
weekday                      0
weekday_int                  0
event                        0
price                        0
revenue                      0
year                         0
month                        0
day                          0
sold_lag_1                   0
sold_lag_2                   0
sold_lag_3                   0
sold_lag_6                   0
sold_lag_12                  0
sold_lag_24                  0
sold_lag_36                  0
item_sold_avg                0
state_sold_avg               0
store_sold_avg               0
cat_sold_avg                 0
dept_sold_avg                0
cat_dept_sold_avg            0
store_item_sold_avg          0
cat_item

In [ ]:
# Salva o DataFrame como um arquivo CSV no Google Drive
#df_final.to_csv("/content/drive/MyDrive/ARQUIVOS TFM/NOVOS DF/df_final_enc.csv", index=False)

## Label Encoding test

In [57]:
#test=pd.read_csv('/content/drive/MyDrive/ARQUIVOS TFM/NOVOS DF/df_final_test.csv')

In [20]:
# Converter as colunas relevantes para categóricas
test['id'] = test['id'].astype('category')
test['item'] = test['item'].astype('category')
test['department'] = test['department'].astype('category')
test['category'] = test['category'].astype('category')
test['store'] = test['store'].astype('category')
test['store_code'] = test['store_code'].astype('category')
test['event'] = test['event'].astype('category')
test['region'] = test['region'].astype('category')
test['weekday'] = test['weekday'].astype('category')

# Armazene as categorias juntamente com seus códigos
d_id = dict(zip(test['id'].cat.codes, test['id']))
d_item_id = dict(zip(test['item'].cat.codes, test['item']))
d_dept_id = dict(zip(test['department'].cat.codes, test['department']))
d_cat_id = dict(zip(test['category'].cat.codes, test['category']))
d_store_id = dict(zip(test['store'].cat.codes, test['store']))
d_state_id = dict(zip(test['region'].cat.codes, test['region']))
d_store_code_id = dict(zip(test['store_code'].cat.codes, test['store_code']))
d_event_id = dict(zip(test['event'].cat.codes, test['event']))
d_weekday_id = dict(zip(test['weekday'].cat.codes, test['weekday']))

In [21]:
directory_path = "/content/drive/MyDrive/ARQUIVOS TFM/mapeamento_lojas/mapeamento_lojas_test"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [22]:
import pickle

def save_mapping(mapping, filename):
    with open(os.path.join(directory_path, filename), "wb") as f:
        pickle.dump(mapping, f)

save_mapping(d_id, "d_id.pkl")
save_mapping(d_item_id, "d_item_id.pkl")
save_mapping(d_dept_id, "d_dept_id.pkl")
save_mapping(d_cat_id, "d_cat_id.pkl")
save_mapping(d_store_id, "d_store_id.pkl")
save_mapping(d_state_id, "d_state_id.pkl")
save_mapping(d_store_code_id, "d_store_code_id.pkl")
save_mapping(d_event_id, "d_event_id.pkl")
save_mapping(d_weekday_id, "d_weekday_id.pkl")


In [23]:
# Crie listas com nomes de colunas e tipos de dados
cols = test.dtypes.index.tolist()
types = test.dtypes.values.tolist()

# Itere pelas colunas e, se for do tipo 'category', converta em códigos
for i, tipo in enumerate(types):
    if tipo.name == 'category':
        test[cols[i]] = test[cols[i]].cat.codes

In [24]:
# Introduzir atrasos
lags = [1, 2, 3, 6, 12, 24, 36]
for lag in lags:
    test[f'sold_lag_{lag}'] = test.groupby(['id', 'item', 'department', 'category', 'store', 'store_code', 'region'], as_index=False)['sold_units'].shift(lag).astype(np.float16)


In [25]:
# Média de vendas por item
test['item_sold_avg'] = test.groupby('item')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por estado
test['state_sold_avg'] = test.groupby('region')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por loja
test['store_sold_avg'] = test.groupby('store_code')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por categoria
test['cat_sold_avg'] = test.groupby('category')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por departamento
test['dept_sold_avg'] = test.groupby('department')['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por categoria e departamento
test['cat_dept_sold_avg'] = test.groupby(['category', 'department'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por loja e item
test['store_item_sold_avg'] = test.groupby(['store_code', 'item'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por categoria e item
test['cat_item_sold_avg'] = test.groupby(['category', 'item'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por departamento e item
test['dept_item_sold_avg'] = test.groupby(['department', 'item'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por estado e loja
test['state_store_sold_avg'] = test.groupby(['region', 'store_code'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por estado, loja e categoria
test['state_store_cat_sold_avg'] = test.groupby(['region', 'store_code', 'category'])['sold_units'].transform('mean').astype(np.float16)

# Média de vendas por loja, categoria e departamento
test['store_cat_dept_sold_avg'] = test.groupby(['store_code', 'category', 'department'])['sold_units'].transform('mean').astype(np.float16)


In [26]:
import numpy as np

window_sizes = [7, 14, 21, 28]

for window_size in window_sizes:
    rolling_col_name = f'rolling_sold_mean_{window_size}'
    test[rolling_col_name] = test.groupby(['id', 'item', 'category', 'department', 'store', 'store_code', 'region'])['sold_units'].transform(lambda x: x.rolling(window=window_size).mean()).astype(np.float16)


In [27]:
# Definindo as colunas para calcular as médias móveis exponenciais acumuladas
columns_to_calculate = ['sold_units', 'revenue']

# Calculando as médias móveis exponenciais acumuladas
for column in columns_to_calculate:
    test[f'expanding_{column}_mean'] = test.groupby(['id', 'item', 'department', 'category', 'store', 'store_code', 'region'])[column].transform(lambda x: x.expanding(2).mean()).astype(np.float16)

In [28]:
# Calcula a média diária de vendas para cada combinação de id, item_id, dept_id, cat_id, store_id, state_id e d
test['daily_avg_sold'] = test.groupby(['id', 'item', 'department', 'category', 'store', 'store_code', 'region', 'd'])['sold_units'].transform('mean').astype(np.float16)

# Calcula a média de vendas geral para cada combinação de id, item_id, dept_id, cat_id, store_id e state_id
test['avg_sold'] = test.groupby(['id', 'item', 'department', 'category', 'store', 'store_code', 'region'])['sold_units'].transform('mean').astype(np.float16)

# Calcula a tendência de vendas subtraindo a média diária de vendas da média de vendas geral
test['selling_trend'] = (test['daily_avg_sold'] - test['avg_sold']).astype(np.float16)

# Remove as colunas daily_avg_sold e avg_sold
test.drop(['daily_avg_sold', 'avg_sold'], axis=1, inplace=True)


In [31]:
#verificando valores em falta
soma_missings_por_coluna = test.isnull().sum()
soma_missings_por_coluna

id                           0
item                         0
category                     0
department                   0
store                        0
store_code                   0
region                       0
d                            0
sold_units                   0
date                         0
weekday                      0
weekday_int                  0
event                        0
price                        0
revenue                      0
year                         0
month                        0
day                          0
sold_lag_1                   0
sold_lag_2                   0
sold_lag_3                   0
sold_lag_6                   0
sold_lag_12                  0
sold_lag_24                  0
sold_lag_36                  0
item_sold_avg                0
state_sold_avg               0
store_sold_avg               0
cat_sold_avg                 0
dept_sold_avg                0
cat_dept_sold_avg            0
store_item_sold_avg          0
cat_item

In [30]:
# Filtrando o dataframe test de volta para os 28 ultimos dias
test = test[test['d'] >= 1886]


In [40]:
# Salva o DataFrame como um arquivo CSV no Google Drive
#test.to_csv("/content/drive/MyDrive/ARQUIVOS TFM/NOVOS DF/test_enc.csv", index=False)

In [ ]:
# Salva o DataFrame como um arquivo CSV no Google Drive
df_final.to_csv("/content/drive/MyDrive/ARQUIVOS TFM/NOVOS DF/df_final_enc.csv", index=False)